## Deep Learning
*Deep learning, as it is primarily used, is essentially a statistical technique for classifying 
patterns, based on sample data, using neural networks with multiple layers*

Source : [Deep Learning: A Critical Appraisal](https://arxiv.org/ftp/arxiv/papers/1801/1801.00631.pdf)

Read these articles for a brief history about the history of neural nets and deep learning.

[part1](http://www.andreykurenkov.com/writing/ai/a-brief-history-of-neural-nets-and-deep-learning/)
[part2](http://www.andreykurenkov.com/writing/ai/a-brief-history-of-neural-nets-and-deep-learning-part-2)
[part3](http://www.andreykurenkov.com/writing/ai/a-brief-history-of-neural-nets-and-deep-learning-part-3)
[part4](http://www.andreykurenkov.com/writing/ai/a-brief-history-of-neural-nets-and-deep-learning-part-4)

The most common use case for deep learning is supervised learning, where once performs the task of matching a predefined set of labels based on the input. Neural nets are very good at this task of mapping inputs to outputs.

The [universal approximation theorem](https://en.wikipedia.org/wiki/Universal_approximation_theorem) states that a simple neural net with a single layer of finite neurons can approximate a wide variety of functions.

### Single layer Neural Net

A simple neural network with one hidden layer would look like this.

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
import numpy as np

output_notebook()

In [ ]:
from bokeh.io import show, output_file
from bokeh.plotting import figure
from bokeh.models import GraphRenderer, StaticLayoutProvider, Circle
from bokeh.models.graphs import NodesAndLinkedEdges, EdgesAndLinkedNodes

num_input = 4
num_hidden = 4
num_output = 1
N = num_input + num_hidden + num_output

node_indices = list(range(N))

plot = figure(title='Neural net with one hidden layer', x_range=(-1,5), y_range=(-1,5))

# graph construction
graph = GraphRenderer()

graph.node_renderer.data_source.add(node_indices, 'index')

# royal blue - #4169E1, darkcyan - #008B8B, orange - #FFA500
colors = ['#4169e1'] * num_input + ['#008b8b'] * num_hidden + ['#ffa500'] * num_output
graph.node_renderer.data_source.add(colors, 'color')
graph.node_renderer.glyph = Circle(size=20, fill_color='color')

input_indices = range(num_input)
hidden_indices = range(num_input, num_input + num_hidden)
edges_input_to_hidden = [(inp, h) for inp in input_indices for h in hidden_indices]

output_indices = range(num_input + num_hidden, N)
edges_hidden_to_output = [(h, o) for h in hidden_indices for o in output_indices]

edges = edges_input_to_hidden + edges_hidden_to_output
start = [x for x, _ in edges]
end = [y for _, y in edges]

graph.edge_renderer.data_source.data = dict(
    start=start,
    end=end)

### start of layout code
input_layer_y = np.arange(num_input)
input_layer_x = np.ones(len(input_layer_y))
hidden_layer_y = np.arange(num_hidden)
hidden_layer_x = 2 * np.ones(len(hidden_layer_y))
output_layer_y = (np.arange(num_output) + max(num_input, num_hidden) - 1) / 2
output_layer_x = 3 * np.ones(len(output_layer_y))

node_x = np.concatenate([input_layer_x, hidden_layer_x, output_layer_x])
node_y = np.concatenate([input_layer_y, hidden_layer_y, output_layer_y])

graph_layout = dict(zip(node_indices, zip(node_x, node_y)))
graph.layout_provider = StaticLayoutProvider(graph_layout=graph_layout)

# add selection, and hover glyphs to edge renderers
# graph.edge_renderer.selection_policy = NodesAndLinkedEdges()
# graph.edge_renderer.inspection_policy = EdgesAndLinkedNodes()

plot.renderers.append(graph)

In [ ]:
show(plot)

The blue layer is the input layer, the inputs simply pass through these nodes. The green layer is the hidden layer. The inputs after passing through the input layer are weighted by a parameter before being passed to the nodes in the hidden layer. The hidden layer aggregates them using a summation and passes the result through a nonlinear function. The process is repeated at all layers that come after the input layer.

For example, in this case, the output of the top neuron in the hidden layer can be written down mathematically as
$$
x_{h1} = \phi(w_1^T x + b_1) \\
x \text{ is the input vector}
$$

Similarly,
$$
x_{h2} = \phi(w_2^T x + b_2) \\
x_{h3} = \phi(w_3^T x + b_3) \\
x_{h4} = \phi(w_4^T x + b_4)
$$

The outputs of the intermediate layer can be aggregated into a single vector as
$$
x_h = \begin{bmatrix}
x_{h1} \\
x_{h2} \\
x_{h3} \\
x_{h4}
\end{bmatrix}
$$

The response of the output layer can then be written as 
$$
x_o = \psi(w_o^T x_h + b_o)
$$

*NOTE* - The choice of activation functions can vary from layer to layer, we will cover some common activation functions later in this notebook.

## Why go deep ?

Although the universal approximation theorem states that a neural net with a single hidden layer is enough, such an architecture may not be computationally tractable. For example, the number of neurons in that single layer might exceed the total number of atoms in the universe or the neural net might take until the end of the universe to converge. We overcome this limitation by going deeper.

Going deep allows you to model more complexity while keeping your parameters from growing at an exponential pace.

Deep learning has become computationally feasible with the advent of modern computing power.

Recommended reading - [Deep Learning Review](http://pages.cs.wisc.edu/~dyer/cs540/handouts/deep-learning-nature2015.pdf)

### Math Background

#### Non-linearity

[Wolfram Mathematica - Linear Functions](http://mathworld.wolfram.com/LinearFunction.html)
 
 A linear function is a function $f:\mathbb{R} \rightarrow \mathbb{R}$ which satisfies

\begin{align*}
f(x+y)=f(x)+f(y)
\end{align*}

and

\begin{align*}
f(\alpha x) &= \alpha f(x) \\
   \forall\  &x, y \in \mathbb{R}, \alpha \in \mathbb{R}. 
\end{align*}

Any function that does not exhibit linearity is a non-linear function. We typically deal with continuous differentiable **non-linear** functions in deep learning.

We will take a closer look at a few examples of common non-linear functions.

### ReLU
The simplest non-linear function is ReLU (rectified linear unit)

\begin{align*}
f(x) &=  \begin{cases}
x & x \geq 0 \\
0 & \text{otherwise}
\end{cases}
\end{align*}

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

x = np.arange(-10, 10, 0.001)

def relu(scalar):
    """
    Rectified Linear Unit (ReLU)
    """
    return scalar if scalar > 0 else 0

y = np.array([relu(i) for i in x])

In [ ]:
from bokeh.plotting import figure
from bokeh.io import show

plot = figure(title='ReLU', x_range=(-10,10), y_range=(-1,10))
plot.line(x,y)
pass

In [ ]:
show(plot)

### Sigmoid / logistic

This is one of the most used activation functions in deep learning (especially in the output layers).

\begin{align*}
f(x) &= \frac{1} {1 + \exp(-x)}
\end{align*}

In [ ]:
x = np.arange(-10, 10, 0.001)

def sigmoid(scalar):
    """
    Rectified Linear Unit (ReLU)
    """
    return 1 / (1 + np.exp(-scalar))

y = np.array([sigmoid(i) for i in x])
sns.set_style("darkgrid")

In [ ]:
from bokeh.plotting import figure
from bokeh.io import show

plot = figure(title='Logistic', x_range=(-10,10), y_range=(-0.5,1.5))
plot.line(x,y)
pass

In [ ]:
show(plot)

#### Loss functions

In ML problems, we evaluate the performance of our model using a [metric](http://mathworld.wolfram.com/Metric.html)/loss function. We will look at some commonly used loss functions.

##### Cross-entropy

A function that closely approximates classification error.

\begin{align*}
K &- \text{number of classes} \\
n &- \text{number of data points} \\
X &- \text{matrix of data points indexed row wise} \\
y_{ik} &- \text{response of the } k^{th} \text{output node to } i^{th} \text{input} \\
CE(X) &= - \sum_{i=1}^{n} \sum_{k=1}^K y_{ik} \log f_k(x_i)
\end{align*}


##### $\mathcal{l}2$ loss

Typically used when the response variable is continuous, e.g. linear regression

\begin{align*}
L(X) &= \sum_{i=1}^{n} (y - f(x))^2
\end{align*}

#### Convex optimization

The choice of loss function greatly influences our solution to the problem. Deep learning problems are posed as minimization of carefully chosen loss functions. It is very important that you pick differentiable continuous convex functions. Non-convex optimization problems are NP-hard.

We tackle convex optimization problems using gradient descent (or its variants). The main idea behind gradient descent to traverse along the opposite direction of the parameter gradient until you hit a minima (which is hopefully global or good enough).

## Linear Regression using Gradient Descent

We use the math we learnt so far in order to solve a linear regression problem. We do this by posing linear regression as a convex optimization problem where the objective is to minimize $l2$ loss.

In [ ]:
# y = 3 x + 5, a = 3, b = 5
x = np.random.uniform(0, 100, size=100)
noise = np.random.normal(0, 15, size=100)

y = 3 * x + 5 + noise

In [ ]:
fig = plt.figure(figsize=(10, 10))
sns.regplot(x,y, ax=fig.gca())

#### Problem formulation (contd.)

We will formulate the problem as a convex optimization problem, and minimize the error using a gradient descent approach.

We will try to minimize the following error/loss/objective function,

\begin{align*}
E(a,b) &= \frac{1}{2N} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2 \\
       &= \frac{1}{2N} \sum_{i=1}^{n} (y_i - ax_i - b )^2 \\
       &= \frac{1}{2N} \sum_{i=1}^{n} y_i^2 + a^2 x_i^2 + b^2 + 2ax_ib - 2ax_iy_i - 2y_ib
\end{align*}

#### Gradient descent

Next, we need to compute the partial derivative w.r.t the parameters `a` and `b`.

\begin{align*}
G(a,b) = \nabla E(a,b) &= \begin{bmatrix}
\frac{\delta E}{\delta a} \\
\frac{\delta E}{\delta b} 
\end{bmatrix} \\
 &= \begin{bmatrix}
 \frac{1}{2N} \sum_{i=1}^{n} 2ax_i^2 + 2bx_i - 2x_iy_i \\
 \frac{1}{2N} \sum_{i=1}^{n} 2b + 2ax_i - 2y_i
 \end{bmatrix} \\
 &= \begin{bmatrix} 
 \frac{1}{2N} \sum_{i=1}^{n} 2x_i (ax_i + b - y_i) \\
 \frac{1}{2N} \sum_{i=1}^{n} 2 (ax_i + b - y_i)
 \end{bmatrix} \\
 &= \begin{bmatrix} 
 \frac{1}{2N} \sum_{i=1}^{n} 2x_i (\hat{y}_i - y_i) \\
 \frac{1}{2N} \sum_{i=1}^{n} 2 (\hat{y}_i - y_i)
 \end{bmatrix} \\
  &= \begin{bmatrix} 
 \frac{1}{N} \sum_{i=1}^{n} x_i \text{(pointwise prediction error)} \\
 \frac{1}{N} \sum_{i=1}^{n} \text{(pointwise prediction error)}
 \end{bmatrix}
\end{align*}

In [ ]:
import numpy as np

def predict(x, a, b):
    """
    Function estimates the response y_hat given input data x, parameters a and b
    """
    ### INSERT YOUR CODE HERE ####
    raise NotImplementedError

def compute_gradient(x, y, a, b):
    """
    Computes gradient in the parameter space.
    """
    ### INSERT YOUR CODE HERE ####
    raise NotImplementedError

In [ ]:
a, b = 0.0, 0.0
learning_rate = 0.0001

def converged(grad_a, grad_b):
    """
    Checks if magnitude of gradient is less than a preset value
    """
    mag_grad = grad_a * grad_a + grad_b * grad_b
    return mag_grad <= 10e-10

grad_a, grad_b = np.inf, np.inf
num_iterations = 0

# normalize your inputs before you perform gradient descent
# or good luck getting it to converge
normalized_x = (x - np.mean(x)) / np.std(x)
normalized_y = (y - np.mean(y)) / np.std(y)

In [ ]:
# DONT RUN THIS CELL REPEATEDLY, DESCENT WILL NOT CONVERGE WITH POOR SEEDS
history_a = []
history_b = []

def rescale(a,b):
    """
    Rescales the coefficients a and b back into the unscaled versions
    """
    b = np.std(y) * (b - a * np.mean(x) / np.std(x)) + np.mean(y)
    a = a * np.std(y) / np.std(x)
    return a ,b

while not converged(grad_a, grad_b) and num_iterations < 10e7:
    ### INSERT YOUR CODE HERE ####
    raise NotImplementedError

# rescale params to the "unnormalized" version
a, b = rescale(a, b)

In [ ]:
print("The parameters for our linear model are a={}, b={}".format(a,b))
errors = (a * x + b) - y
print("Root mean squared error (RMSE) : ", np.sqrt(np.mean(np.power(errors, 2))))

In [ ]:
import matplotlib.pyplot as plt
resp_y = a * x + b
fig = plt.figure(figsize=(10,8))
plt.scatter(x,y)
plt.plot(x, resp_y, color='red')
plt.show()

#### Conclusion

The manual approach to computing gradients is error prone and not easy for modern neural nets. For example, the VGG16 convnet architecture has 14 million trainable parameters (these nets are better than humans at performing large scale object recognition).

This is why we use tools like TensorFlow and Keras

### Tools
<br/>
<center>
<img src=Tensorflow_logo.svg height=200/>
<br>
<img src=keras-logo-small-2018.jpg height=200 width=100/>
</center>

#### TensorFlow

A matrix is a 2D ordered collection.
A tensor is a generalization of a matrix to `n` dimensions, i.e. a tensor is a `n` dimensional ordered collection.

TensorFlow is an API that makes it relatively easy to manipulate tensors. TensorFlow is suited for deep learning problems since it does a lot of things for the developer automatically. Models are constructed in the form of computational graphs, automatic differentiation using a subgraph makes it easier to do things like backpropagation with very little input from the developer.

TensorFlow is open-sourced by Google, the website has a lot of tutorials and documentation regarding the use of TensorFlow. For further reading and insight into the inner workings of TensorFlow, we recommend the following links.

   - [Automatic differentiation](http://www.columbia.edu/~ahd2125/post/2015/12/5)
   - [Low level intro](https://www.tensorflow.org/programmers_guide/low_level_intro)
   - [Programmer's guide](https://www.tensorflow.org/programmers_guide)

TensorFlow is aimed at building low level computational graphs (although it does have high level modules in contrib), which is why we build using Keras.

#### Keras

From the keras [website](https://keras.io),

*Keras is a high-level neural networks API, written in Python and capable of running on top of TensorFlow, CNTK, or Theano. It was developed with a focus on enabling fast experimentation. Being able to go from idea to result with the least possible delay is key to doing good research.*

*Use Keras if you need a deep learning library that*

   - *Allows for easy and fast prototyping (through user friendliness, modularity, and extensibility).*
   - *Supports both convolutional networks and recurrent networks, as well as combinations of the two.*
   - *Runs seamlessly on CPU and GPU.*

Using Keras, we can prototype neural nets fast without a lot of knowledge regarding construction of computational graphs. Keras also supports other low level libraries like Theano and CNTK.

#### TensorBoard

TensorBoard is a debugging tool that goes with TensorFlow, it is useful in visualizing your results.
We will be using it to monitor our training runs.

### Iris classification

Read more about the dataset on [Wikipedia](https://en.wikipedia.org/wiki/Iris_flower_data_set). This is a very old dataset with 150 data points.

The data set consists of measurements of the three species of the iris flower(iris setosa, iris virginica and iris versicolor)

In [ ]:
import pandas as pd
import seaborn as sns

iris = sns.load_dataset("iris")
print("Dataset has {} data points".format(len(iris)))
iris.head()

In [ ]:
sns.pairplot(iris, hue="species", size=2)

In [ ]:
X = iris[["sepal_length", "sepal_width", "petal_length", "petal_width"]]
y = iris["species"]

X_mean = X.apply(np.mean, axis=0)
X_std  = X.apply(np.std,  axis=0)
X_norm = (X - X_mean) / X_std
y_label = pd.get_dummies(y)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()

### INSERT YOUR CODE HERE ####
raise NotImplementedError

In [ ]:
import os
from keras import backend as K
from lib.default_utils import default_callbacks

K.set_learning_phase(True)  # important if you have modules like dropout in your model

### INSERT YOUR CODE HERE ####
raise NotImplementedError

In [ ]:
loss, score = model.evaluate(X_norm, y_label)
print("This model has an accuracy of {}".format(score * 100))